# 8.2

---

## 8.2.1.1

In [ ]:
import requests as rq   # 웹페이지 요청 라이브러리

url = 'https://quotes.toscrape.com/'
quote = rq.get(url)

quote.content[:1000]

b'<!DOCTYPE html>\n<html lang="en">\n<head>\n\t<meta charset="UTF-8">\n\t<title>Quotes to Scrape</title>\n    <link rel="stylesheet" href="/static/bootstrap.min.css">\n    <link rel="stylesheet" href="/static/main.css">\n    \n    \n</head>\n<body>\n    <div class="container">\n        <div class="row header-box">\n            <div class="col-md-8">\n                <h1>\n                    <a href="/" style="text-decoration: none">Quotes to Scrape</a>\n                </h1>\n            </div>\n            <div class="col-md-4">\n                <p>\n                \n                    <a href="/login">Login</a>\n                \n                </p>\n            </div>\n        </div>\n    \n\n<div class="row">\n    <div class="col-md-8">\n\n    <div class="quote" itemscope itemtype="http://schema.org/CreativeWork">\n        <span class="text" itemprop="text">\xe2\x80\x9cThe world as we have created it is a process of our thinking. It cannot be changed without changing our thinki

In [ ]:
from bs4 import BeautifulSoup   # html 형식으로 뽑을 수 있게 잘 바꿔주는 라이브러리

quote_html = BeautifulSoup(quote.content, 'html.parser')
quote_html.head()

[<meta charset="utf-8"/>,
 <title>Quotes to Scrape</title>,
 <link href="/static/bootstrap.min.css" rel="stylesheet"/>,
 <link href="/static/main.css" rel="stylesheet"/>]

In [8]:
# 명언이 존재하는 div 태그를 먼저 뽑음

quote_div = quote_html.find_all('div', class_='quote')

quote_div[0]

<div class="quote" itemscope="" itemtype="http://schema.org/CreativeWork">
<span class="text" itemprop="text">“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”</span>
<span>by <small class="author" itemprop="author">Albert Einstein</small>
<a href="/author/Albert-Einstein">(about)</a>
</span>
<div class="tags">
            Tags:
            <meta class="keywords" content="change,deep-thoughts,thinking,world" itemprop="keywords"/>
<a class="tag" href="/tag/change/page/1/">change</a>
<a class="tag" href="/tag/deep-thoughts/page/1/">deep-thoughts</a>
<a class="tag" href="/tag/thinking/page/1/">thinking</a>
<a class="tag" href="/tag/world/page/1/">world</a>
</div>
</div>

In [9]:
# 명언이 존재하는 div 태그 내의 span 태그를 따로 뽑기

quote_span = quote_div[0].find_all('span', class_='text')

quote_span

[<span class="text" itemprop="text">“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”</span>]

In [15]:
# span 태그 내의 텍스트 데이터만 추출

quote_span[0].text

'“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”'

In [16]:
# 위의 과정을 한번에 처리하기

quote_div = quote_html.find_all('div', class_='quote')

[i.find_all('span', class_='text')[0].text for i in quote_div]

['“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”',
 '“It is our choices, Harry, that show what we truly are, far more than our abilities.”',
 '“There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.”',
 '“The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.”',
 "“Imperfection is beauty, madness is genius and it's better to be absolutely ridiculous than absolutely boring.”",
 '“Try not to become a man of success. Rather become a man of value.”',
 '“It is better to be hated for what you are than to be loved for what you are not.”',
 "“I have not failed. I've just found 10,000 ways that won't work.”",
 "“A woman is like a tea bag; you never know how strong it is until it's in hot water.”",
 '“A day without sunshine is like, you know, night.”']

---

## 8.2.1.2

In [21]:
# select 함수 내에 찾고자 하는 태그를 입력, 클래스명이 존재할 경우 . 이후에 클래스명 입력
# 여러 태그를 찾아 내려갈 때는 > 사용
quote_text = quote_html.select('div.quote > span.text')

quote_text

[<span class="text" itemprop="text">“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”</span>,
 <span class="text" itemprop="text">“It is our choices, Harry, that show what we truly are, far more than our abilities.”</span>,
 <span class="text" itemprop="text">“There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.”</span>,
 <span class="text" itemprop="text">“The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.”</span>,
 <span class="text" itemprop="text">“Imperfection is beauty, madness is genius and it's better to be absolutely ridiculous than absolutely boring.”</span>,
 <span class="text" itemprop="text">“Try not to become a man of success. Rather become a man of value.”</span>,
 <span class="text" itemprop="text">“It is better to be hated for what you are than to be loved for what you are not.

In [20]:
quote_text_list = [i.text for i in quote_text]

quote_text_list

['“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”',
 '“It is our choices, Harry, that show what we truly are, far more than our abilities.”',
 '“There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.”',
 '“The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.”',
 "“Imperfection is beauty, madness is genius and it's better to be absolutely ridiculous than absolutely boring.”",
 '“Try not to become a man of success. Rather become a man of value.”',
 '“It is better to be hated for what you are than to be loved for what you are not.”',
 "“I have not failed. I've just found 10,000 ways that won't work.”",
 "“A woman is like a tea bag; you never know how strong it is until it's in hot water.”",
 '“A day without sunshine is like, you know, night.”']

In [25]:
# 말한 사람 이름 추출하기
quote_author = quote_html.select('div.quote > span > small.author')

quote_author_list = [i.text for i in quote_author]
quote_author_list

['Albert Einstein',
 'J.K. Rowling',
 'Albert Einstein',
 'Jane Austen',
 'Marilyn Monroe',
 'Albert Einstein',
 'André Gide',
 'Thomas A. Edison',
 'Eleanor Roosevelt',
 'Steve Martin']

In [26]:
# 말한 사람 정보 추출하기
quote_link = quote_html.select('div.quote > span > a')

quote_link

[<a href="/author/Albert-Einstein">(about)</a>,
 <a href="/author/J-K-Rowling">(about)</a>,
 <a href="/author/Albert-Einstein">(about)</a>,
 <a href="/author/Jane-Austen">(about)</a>,
 <a href="/author/Marilyn-Monroe">(about)</a>,
 <a href="/author/Albert-Einstein">(about)</a>,
 <a href="/author/Andre-Gide">(about)</a>,
 <a href="/author/Thomas-A-Edison">(about)</a>,
 <a href="/author/Eleanor-Roosevelt">(about)</a>,
 <a href="/author/Steve-Martin">(about)</a>]

In [27]:
# quote_link[0]['href'] - 속성 값 정보 추출

['https://quotes.toscrape.com' + i['href'] for i in quote_link]

['https://quotes.toscrape.com/author/Albert-Einstein',
 'https://quotes.toscrape.com/author/J-K-Rowling',
 'https://quotes.toscrape.com/author/Albert-Einstein',
 'https://quotes.toscrape.com/author/Jane-Austen',
 'https://quotes.toscrape.com/author/Marilyn-Monroe',
 'https://quotes.toscrape.com/author/Albert-Einstein',
 'https://quotes.toscrape.com/author/Andre-Gide',
 'https://quotes.toscrape.com/author/Thomas-A-Edison',
 'https://quotes.toscrape.com/author/Eleanor-Roosevelt',
 'https://quotes.toscrape.com/author/Steve-Martin']

---

## 8.2.1.3

In [28]:
# 웹사이트의 모든 페이지를 크롤링하는 예제

import requests as rq
from bs4 import BeautifulSoup
import time

text_list = []
author_list = []
infor_list = []

for i in range(1, 100):
    url = f'https://quotes.toscrape.com/page/{i}/'
    quote = rq.get(url)
    quote_html = BeautifulSoup(quote.content, 'html.parser')
    
    quote_text = quote_html.select('div.quote > span.text')
    quote_text_list = [i.text for i in quote_text]
    
    quote_author = quote_html.select('div.quote > span > small.author')
    quote_author_list = [i.text for i in quote_author]
    
    quote_link = quote_html.select('div.quote > span > a')
    quote_link_list = ['https://quotes.toscrape.com' + i['href'] for i in quote_link]
    
    if len(quote_text_list) > 0:
        text_list.extend(quote_text_list)
        author_list.extend(quote_author_list)
        infor_list.extend(quote_link_list)
        time.sleep(1)
        
    else:
        break

In [30]:
import pandas as pd

quote_df = pd.DataFrame({'text': text_list, 'author': author_list, 'infor': infor_list})
quote_df

,text,author,infor
0,“The world as we have created it is a process ...,Albert Einstein,https://quotes.toscrape.com/author/Albert-Eins...
1,"“It is our choices, Harry, that show what we t...",J.K. Rowling,https://quotes.toscrape.com/author/J-K-Rowling
2,“There are only two ways to live your life. On...,Albert Einstein,https://quotes.toscrape.com/author/Albert-Eins...
3,"“The person, be it gentleman or lady, who has ...",Jane Austen,https://quotes.toscrape.com/author/Jane-Austen
4,"“Imperfection is beauty, madness is genius and...",Marilyn Monroe,https://quotes.toscrape.com/author/Marilyn-Monroe
...,...,...,...
95,“You never really understand a person until yo...,Harper Lee,https://quotes.toscrape.com/author/Harper-Lee
96,“You have to write the book that wants to be w...,Madeleine L'Engle,https://quotes.toscrape.com/author/Madeleine-L...
97,“Never tell the truth to people who are not wo...,Mark Twain,https://quotes.toscrape.com/author/Mark-Twain
98,"“A person's a person, no matter how small.”",Dr. Seuss,https://quotes.toscrape.com/author/Dr-Seuss


---

## 8.2.2


In [ ]:
# 기사 제목 뽑기 예제
import requests as rq
from bs4 import BeautifulSoup

url = 'https://finance.naver.com/news/news_list.nhn?mode=LSS2D&section_id=101&section_id2=258'
data = rq.get(url)
html = BeautifulSoup(data.content, 'html.parser')
html_select = html.select('dl > dd.articleSubject > a')

html_select[0:3]

[<a href="/news/news_read.naver?article_id=0005423701&amp;office_id=009&amp;mode=LSS2D&amp;type=0§ion_id=101§ion_id2=258§ion_id3=&amp;date=20250104&amp;page=1" title="“그래도 10조는 넘어설 줄 알았는데”…삼성전자 4분기 영업익 얼마길래">“그래도 10조는 넘어설 줄 알았는데”…삼성전자 4분기 영업익 얼마길래</a>,
 <a href="/news/news_read.naver?article_id=0000032131&amp;office_id=648&amp;mode=LSS2D&amp;type=0§ion_id=101§ion_id2=258§ion_id3=&amp;date=20250104&amp;page=1" title="삼성 스마트싱스, 작년 2천만명이 썼다">삼성 스마트싱스, 작년 2천만명이 썼다</a>,
 <a href="/news/news_read.naver?article_id=0005077889&amp;office_id=015&amp;mode=LSS2D&amp;type=0§ion_id=101§ion_id2=258§ion_id3=&amp;date=20250104&amp;page=1" title="KB운용 김영성 대표, 임기 마지막해 '군살 빼기' 나선 이유 [돈앤톡]">KB운용 김영성 대표, 임기 마지막해 '군살 빼기' 나선 이유 [돈앤톡]</a>]

In [33]:
# title 속성에 있는 내용 뽑아내기
[i['title'] for i in html_select]

['“그래도 10조는 넘어설 줄 알았는데”…삼성전자 4분기 영업익 얼마길래',
 '삼성 스마트싱스, 작년 2천만명이 썼다',
 "KB운용 김영성 대표, 임기 마지막해 '군살 빼기' 나선 이유 [돈앤톡]",
 '삼성SDS "국내 최고 클라우드 사업자로 공인"',
 "2025년 기업 신년사의 공통 주제는 '위기'",
 '"환율 오를대로 올랐다"…국민연금, 80조 전략적 환헤지 임박',
 '金투자 다 같은 ‘金’이 아니었다…투자 방법 따라 수익률 2배?[투자360]',
 '바이든, 일본제철의 US스틸 인수 불허…日정부 "유감"',
 '[뉴욕 증시] 3대 지수, 동반 강세 마감…저가 매수+美제조업 지수 개선',
 '가계대출 증가액 1조 아래로 꺾였다…새해 대출 문턱 내려간다',
 '반등하는 코스피…외국인 돌아오나',
 '"올해도 두 배 뛴다고?"…트럼프 취임에 부푸는 비트코인 [투자의 재발견]',
 '회계 부정 의혹 슈퍼마이크로 간만에 11% 폭등',
 '모처럼 웃은 코스피…삼성전자 실적에 쏠린 눈[다음 주 증시 전망]',
 '[청기홍기]높은 운임에 장기계약 체결한 현대글로비스…목표가↑',
 '[IPO챗] LG CNS 등 7개사 신규 수요예측 돌입…공모주 투심 풀릴까',
 '비트코인, 지금이 제일 싼 가격?… 9만8000달러 재돌파',
 '[다음주 경제] 경상수지, 7개월 연속 흑자 이어졌나',
 '삼성전자까지 뛰어들더니…이 종목, 이틀새 47% 뛰었다 [종목+]',
 '비트코인 10만달러 재돌파 임박?… 최고가의 91%까지 회복']

---

## 8.2.3

In [37]:
# 크롤링을 원하는 데이터가 표 형태로 되어있을 경우

import pandas as pd

url = 'https://en.wikipedia.org/wiki/List_of_countries_by_stock_market_capitalization'
tbl = pd.read_html(url)

tbl[0].head()

Country / Territory        Total market cap                     \
  Country / Territory (in millions of US$)[3] (as a % of GDP)[4]   
0       United States                54000000              194.5   
1               China           12,009,242[5]               65.1   
2               Japan            6,245,880[7]           146.2[8]   
3               India            5,663,221[9]              140.1   
4           Hong Kong           4,746,241[11]             1262.6   

  Number of domestic companies listed[2]  Year  
  Number of domestic companies listed[2]  Year  
0                                   4642  2024  
1                               5,363[6]  2024  
2                                   3865  2024  
3                              5,451[10]  2024  
4                                   2414  2024

---

## 8.2.4

In [ ]:
# 한국거래소 상장공시시스템에서 POST 요청을 보내 데이터 뽑아내기

import requests as rq
from bs4 import BeautifulSoup
import pandas as pd

url = 'https://kind.krx.co.kr/disclosure/todaydisclosure.do'
payload = {
   'method': 'searchTodayDisclosureSub',
   'currentPageSize': '15',
   'pageIndex': '1',
   'orderMode': '0',
   'orderStat':'D',
   'forward': 'todaydisclosure_sub',
   'chose': 'S',
   'todayFlag': 'N',
   'selDate': '2025-01-03'
}

data = rq.post(url, data=payload)       # post request
html = BeautifulSoup(data.content, 'html.parser')

# print(html)


<section class="scrarea type-00">
<table class="list type-00 mt10" summary="시간, 회사명, 공시제목, 제출인, 차트/주가">
<caption>목록</caption>
<colgroup>
<col width="9%"/>
<col width="22%"/>
<col width="*"/>
<col width="16%"/>
<col width="9%"/>
</colgroup>
<thead>
<tr class="first active" id="title-contents">
</tr>
</thead>
<tbody>
<tr class="first" id="parkman">
<td class="first txc">18:25</td>
<td><img alt="유가증권" class="vmiddle legend" src="/images/common/icn_t_yu.gif"/> <a href="#companysum" id="companysum" onclick="companysummary_open('00392'); return false;" title="남양유업"> 남양유업</a> </td>
<td><a href="#viewer" onclick="openDisclsViewer('20250103000761','')" title="횡령ㆍ배임혐의발생"><font color="#FF8040">[정정]</font>횡령ㆍ배임혐의발생</a></td>
<td>남양유업</td>
<td class="txc">
<a class="btn ico chart-00" href="#" onclick="openDisclsChart('00392');return false;" title="공시차트"><span>공시차트</span></a>
<a class="btn ico chart-01" href="#" onclick="fnPopStockPrices('00392');return false;" title="주가차트"><span>주가차트</span></a>
</t

In [41]:
html_unicode = html.prettify()  # BeautifulSoup에서 파싱한 파서 트리를 유니코드 형태로 돌려놓기
html_unicode

'<section class="scrarea type-00">\n <table class="list type-00 mt10" summary="시간, 회사명, 공시제목, 제출인, 차트/주가">\n  <caption>\n   목록\n  </caption>\n  <colgroup>\n   <col width="9%"/>\n   <col width="22%"/>\n   <col width="*"/>\n   <col width="16%"/>\n   <col width="9%"/>\n  </colgroup>\n  <thead>\n   <tr class="first active" id="title-contents">\n   </tr>\n  </thead>\n  <tbody>\n   <tr class="first" id="parkman">\n    <td class="first txc">\n     18:25\n    </td>\n    <td>\n     <img alt="유가증권" class="vmiddle legend" src="/images/common/icn_t_yu.gif"/>\n     <a href="#companysum" id="companysum" onclick="companysummary_open(\'00392\'); return false;" title="남양유업">\n      남양유업\n     </a>\n    </td>\n    <td>\n     <a href="#viewer" onclick="openDisclsViewer(\'20250103000761\',\'\')" title="횡령ㆍ배임혐의발생">\n      <font color="#FF8040">\n       [정정]\n      </font>\n      횡령ㆍ배임혐의발생\n     </a>\n    </td>\n    <td>\n     남양유업\n    </td>\n    <td class="txc">\n     <a class="btn ico chart-00" href="#" 

In [44]:
tbl = pd.read_html(html.prettify()) # html 문자열로 변환 후 read_html 사용
tbl[0].head()

C:\Users\palsc\AppData\Local\Temp\ipykernel_49384\1181271816.py:1: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tbl = pd.read_html(html.prettify())


,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,18:25,남양유업,[정정] 횡령ㆍ배임혐의발생,남양유업,공시차트 주가차트
1,18:10,NaN,일일 회전율 상위 20종목-ELW제외(유가증권시장),유가증권시장본부,NaN
2,18:10,NaN,일일 회전율 상위 20종목-ELW포함(유가증권시장),유가증권시장본부,NaN
3,18:10,NaN,대량매매내역(유가증권시장),유가증권시장본부,NaN
4,18:10,NaN,자기주식매매 신청내역(유가증권시장),유가증권시장본부,NaN
